In [16]:
import pymysql.cursors

connection = None

def get_connection(): # Подключение к базе данных (если еще не подключено)
    global connection
    if connection is None:
        connection = pymysql.connect(
            host='ich-db.ccegls0svc9m.eu-central-1.rds.amazonaws.com',
            user='ich1',
            password='password',
            database='ich_edit',
            cursorclass=pymysql.cursors.DictCursor
        )
    return connection

# Функция для выбора столбцов из таблицы
def input_columns(table):
    columns = {
        "users": ["id", "name", "age"],
        "product": ["pid", "prod", "quantity"],
        "sales": ["sid", "pid", "id"]
    }
    field = input(f"Выберите одно из полей {columns[table]} этой таблицы или введите * для выбора всех: ").strip()
    if field not in columns[table] and field != "*":
        raise ValueError("Введено некорректное значение. Будет использовано значение '*'.")
    return field

# Функция для выбора знака сравнения
def input_comparison_operator():
    operators = [">", "<", "="]
    operator = input(f"Выберите знак сравнения ({', '.join(operators)}): ").strip()
    if operator not in operators:
        raise ValueError("Введено некорректное значение. Будет использован знак '='.")
    return operator

# Функция для фильтрации по значению
def input_filter_value():
    value = input("Введите значение для фильтрации: ").strip()
    return value

# Функция для выбора таблиц и формирования запроса
def select_tables():
    valid_tables = {"users", "product", "sales"}
    str_of_tables = ", ".join(valid_tables)
    tables = input(f"Введите таблицу или список таблиц из набора {str_of_tables}: ").split()

    # Проверка на корректность ввода
    if not set(tables).issubset(valid_tables):
        raise ValueError("Введены некорректные таблицы.")

    # Проверка на несовместимость таблиц
    if set(tables) == {"users", "product"}:
        raise ValueError("Эти таблицы не связаны. Сделайте другой выбор.")

    # Формируем запрос в зависимости от выбранных таблиц
    if len(tables) == 1:
        table = tables[0]
        field = input_columns(table)

        # Если выбрано конкретное поле, добавляем фильтрацию
        if field != "*":
            operator = input_comparison_operator()
            value = input_filter_value()
            query = f"SELECT {field} FROM {table} WHERE {field} {operator} %s"
            params = (value,)  # Инициализация params
        else:
            query = f"SELECT {field} FROM {table}"
            params = None

    elif "users" in tables and "sales" in tables and "product" in tables:
        query = """
            SELECT users.id, users.name, users.age, product.prod, product.quantity
            FROM sales
            JOIN users ON sales.id = users.id
            JOIN product ON sales.pid = product.pid
        """
        params = None
    elif "users" in tables and "sales" in tables:
        query = """
            SELECT users.id, users.name, users.age
            FROM sales
            JOIN users ON sales.id = users.id
        """
        params = None
    elif "sales" in tables and "product" in tables:
        query = """
            SELECT product.pid, product.prod, product.quantity
            FROM sales
            JOIN product ON sales.pid = product.pid
        """
        params = None
    else:
        raise ValueError("Невозможно выполнить запрос для выбранных таблиц.")

    return query, params

# Основной блок программы

try:
    conn = get_connection()
    query, params = select_tables()

    with conn.cursor() as cursor:
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)

        result = cursor.fetchall()
        if result:
            for row in result:
                print(row)
        else:
            print("Записи не найдены.")
except ValueError as e:
    print(f"Ошибка: {e}")
except Exception as e:
    print(f"Произошла ошибка: {e}")



Введите таблицу или список таблиц из набора users, sales, product:  users
Выберите одно из полей ['id', 'name', 'age'] этой таблицы или введите * для выбора всех:  age
Выберите знак сравнения (>, <, =):  >
Введите значение для фильтрации:  60


{'age': 67}
{'age': 88}
{'age': 99}
{'age': 98}
{'age': 100}
{'age': 82}
{'age': 82}
{'age': 70}
{'age': 82}
{'age': 80}
{'age': 100}
{'age': 94}
{'age': 75}
{'age': 75}
{'age': 75}
{'age': 78}
{'age': 78}
